In [ ]:
!pip install streamlit spacy langdetect plotly pymupdf python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993221 sha256=32ca7e6e78ab4a3b3792841b72f04014b0ddef20840add7c9875842d116f7af1
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [ ]:
!wget -q -O - ipv4.icanhazip.com

35.225.34.210


In [ ]:
!npm install localtunnel


up to date, audited 23 packages in 476ms

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.


In [ ]:
%%writefile edu_enhanceapp.py
import streamlit as st
import spacy
from langdetect import detect
from collections import Counter
import plotly.graph_objects as go
import random
import fitz
import docx

# Load the English NLP model
nlp = spacy.load("en_core_web_sm")

# Mock user credentials for login
user_credentials = {
    "Aswath": "aswath@123"
}

# Function to extract text from PDF file
def extract_text_from_pdf(pdf_file):
    pdf_document = fitz.open(stream=pdf_file.read(), filetype="pdf")
    text = ""
    for page in pdf_document:
        text += page.get_text()
    return text

# Function to extract text from DOCX file
def extract_text_from_docx(docx_file):
    doc = docx.Document(docx_file)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return "\n".join(full_text)

# Function to simulate plagiarism check.
def check_plagiarism(text):
    return random.randint(0, 100)

# Function to detect redundancy in text
def detect_redundancy(doc):
    words = [token.text.lower() for token in doc if token.is_alpha]
    word_count = Counter(words)
    redundant_words = [word for word, count in word_count.items() if count > 3]
    return redundant_words

# Function to analyze coherence
def analyze_coherence(doc):
    paragraphs = [p for p in doc.text.split('\n') if p.strip()]
    if len(paragraphs) < 2:
        return "Try expanding your content with more paragraphs for better coherence."
    return "The paragraph structure looks coherent."

# Function to personalize feedback based on user level
def personalize_feedback(score, user_level):
    feedback = []
    if user_level == "beginner":
        feedback.append("You're off to a good start! Focus on improving sentence structure and avoid redundancy.")
    elif user_level == "intermediate":
        feedback.append("Well done! Try refining your content further by using more advanced vocabulary and clearer structure.")
    elif user_level == "advanced":
        feedback.append("Excellent work! Your content is very polished. Focus on minor improvements for a flawless result.")
    return feedback

# AI-Driven content scoring
def content_scoring(doc, user_level):
    score = 100
    feedback = []

    if not all([sent.text.endswith('.') for sent in doc.sents]):
        score -= 10
        feedback.append("Some sentences are missing periods.")

    redundant_words = detect_redundancy(doc)
    if redundant_words:
        score -= 10
        feedback.append(f"Redundant words found: {', '.join(redundant_words)}")

    coherence_feedback = analyze_coherence(doc)
    if "Try expanding" in coherence_feedback:
        score -= 10
        feedback.append(coherence_feedback)

    feedback.extend(personalize_feedback(score, user_level))
    if score == 100:
        feedback.append("Excellent! Your content is well-written.")
    else:
        feedback.append(f"Your content scored {score}/100.")

    return score, feedback

# Multilingual detection
def detect_language(text):
    try:
        lang = detect(text)
    except:
        lang = "en"  # Default to English if detection fails
    return lang

# Function to analyze text and generate feedback
def analyze_text(user_input, user_level):
    lang = detect_language(user_input)

    if lang != 'en':
        return f"Currently, feedback is only available in English. Detected language: {lang}"

    doc = nlp(user_input)
    feedback = []

    for sent in doc.sents:
        if not sent.text.endswith('.'):
            feedback.append(f"Sentence should end with a period: \"{sent.text}\"")

    redundant_words = detect_redundancy(doc)
    if redundant_words:
        feedback.append(f"Redundant words detected: {', '.join(redundant_words)}.")

    feedback.append(analyze_coherence(doc))
    score, scoring_feedback = content_scoring(doc, user_level)
    feedback.extend(scoring_feedback)

    return score, "\n".join(feedback)

# Function to create pie chart
def plot_pie_chart(value, label):
    labels = [label, "Remaining"]
    values = [value, 100 - value]

    fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.3)])
    fig.update_layout(
        title=f"{label} Percentage",
        template="plotly_white"
    )
    return fig

# Function to assign a grade based on score
def assign_grade(score):
    if score >= 90:
        return "O"
    elif score >= 80:
        return "A"
    elif score >= 70:
        return "B"
    elif score >= 60:
        return "C"
    else:
        return "D"

# Main dashboard
def create_dashboard(user_input, user_level):
    score, feedback = analyze_text(user_input, user_level)
    plagiarism_percent = check_plagiarism(user_input)

    plagiarism_chart = plot_pie_chart(plagiarism_percent, "Plagiarism Detected")
    content_score_chart = plot_pie_chart(score, "Content Score")

    grade = assign_grade(score)

    return feedback, plagiarism_chart, content_score_chart, grade

# App starts here
st.title("EduEnhance Dashboard")

# Authentication
if "authenticated" not in st.session_state:
    st.session_state.authenticated = False

if not st.session_state.authenticated:
    st.subheader("Login")

    username = st.text_input("Username")
    password = st.text_input("Password", type="password")

    if st.button("Login"):
        if username in user_credentials and user_credentials[username] == password:
            st.session_state.authenticated = True
            st.success("Login successful!")
        else:
            st.error("Invalid username or password.")

# After login
if st.session_state.authenticated:
    st.subheader("Upload a File or Enter Text")

    # File upload options
    file = st.file_uploader("Choose a file", type=["pdf", "docx"])
    user_input = None

    if file is not None:
        # If it's a PDF, extract text
        if file.type == "application/pdf":
            user_input = extract_text_from_pdf(file)
        # If it's a DOCX, extract text
        elif file.type == "application/vnd.openxmlformats-officedocument.wordprocessingml.document":
            user_input = extract_text_from_docx(file)

        if user_input:
            st.text_area("Extracted Text", value=user_input, height=200)

    else:
        user_input = st.text_area("Or Enter Your Text", height=200)

    user_level = st.radio("Select Your Level", ["beginner", "intermediate", "advanced"])

    if st.button("Submit") and user_input:
        feedback, plagiarism_chart, content_score_chart, grade = create_dashboard(user_input, user_level)

        st.subheader("Feedback")
        st.text(feedback)

        st.subheader("Plagiarism Detection")
        st.plotly_chart(plagiarism_chart)

        st.subheader("Content Score")
        st.plotly_chart(content_score_chart)

        st.subheader("Grade")
        st.text(f"Your grade is: {grade}")

Overwriting edu_enhanceapp.py


In [ ]:
!wget -q -O - ipv4.icanhazip.com

35.225.34.210


In [ ]:
!streamlit run edu_enhanceapp.py &>/dev/null&
!npx localtunnel --port 8501

your url is: https://blue-baboons-thank.loca.lt
